In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx

In [26]:
data_file = open('../data/Wiki-Vote.txt', 'r')
lines = data_file.readlines()

In [27]:
start_ptr = 0
for i in range(len(lines)):
    if(lines[i][0] == '#'):
        start_ptr += 1

In [28]:
node_set = set()
for i in range(start_ptr, len(lines)):
    split1 = lines[i].split('\t')
    node1 = int(split1[0])
    node2 = int(split1[1][:(len(split1[1]) - 1)])
    node_set.add(node1)
    node_set.add(node2)

In [29]:
num_nodes = len(node_set)

In [30]:
node_num = 0
node_id_to_num = {}
node_num_to_id = {}
for node_id in node_set:
    node_id_to_num[node_id] = node_num
    node_num_to_id[node_num] = node_id
    node_num += 1

In [31]:
# adjacency_matrix = np.zeros((num_nodes, num_nodes))
edge_list = []
for i in range(start_ptr, len(lines)):
    split1 = lines[i].split('\t')
    node1_id = int(split1[0])
    node2_id = int(split1[1][:(len(split1[1]) - 1)])
    node1_num = node_id_to_num[node1_id]
    node2_num = node_id_to_num[node2_id]
    edge_list.append([node1_num, node2_num])
    # adjacency_matrix[node1_num][node2_num] = 1

Forming Graph

In [32]:
G = nx.DiGraph() #making a directed graph from edge list
G.add_nodes_from(node_id_to_num.values())
G.add_edges_from(edge_list)

Pagerank Scores

In [33]:
pagerank_scores = nx.pagerank(G) #computing pagerank scores 

In [34]:
print(f"PageRank Scores :\n{pagerank_scores}")

PageRank Scores :
{0: 0.00020539498232448016, 1: 5.048782345863015e-05, 2: 5.048782345863015e-05, 3: 0.00031183250978437455, 4: 5.048782345863015e-05, 5: 0.00032663557615950425, 6: 5.048782345863015e-05, 7: 0.0004213996615598798, 8: 5.048782345863015e-05, 9: 5.048782345863015e-05, 10: 5.048782345863015e-05, 11: 5.048782345863015e-05, 12: 0.00368122072952927, 13: 5.048782345863015e-05, 14: 5.048782345863015e-05, 15: 5.048782345863015e-05, 16: 0.00013112179292607272, 17: 5.048782345863015e-05, 18: 5.048782345863015e-05, 19: 5.048782345863015e-05, 20: 0.00017122390637420328, 21: 5.048782345863015e-05, 22: 5.048782345863015e-05, 23: 5.048782345863015e-05, 24: 5.048782345863015e-05, 25: 0.0016986730322136924, 26: 0.00018490986415744417, 27: 0.00017349553934328335, 28: 5.048782345863015e-05, 29: 0.00019385458408414878, 30: 0.0003386160040196025, 31: 0.0002606886538476849, 32: 0.0007007673625519543, 33: 0.00013857302939342643, 34: 5.048782345863015e-05, 35: 0.00016519399800671061, 36: 0.00034

Hubs and Authorities Scores

In [35]:
hits_scores = nx.hits(G) #HITS algor
hubs_scores = hits_scores[0] #hubs_scores
authorities_scores = hits_scores[1] #authorities score

In [36]:
print(f"Hubs Scores :\n{hubs_scores}")

Hubs Scores :
{0: 4.021031639777643e-05, 1: 7.319607685824188e-05, 2: 3.5017884744336435e-05, 3: 0.0010539872861763625, 4: 8.200618013274951e-05, 5: 0.00032001233330871515, 6: 0.00023181963355171467, 7: 0.00018207047608178467, 8: 0.004921182063808107, 9: 0.0002882286245176333, 10: 3.1200539556819355e-05, 11: 0.0004975871202612588, 12: 0.00041573260338093835, 13: 8.409812451935368e-05, 14: 0.00010084393686112614, 15: 3.617008656019807e-05, 16: 7.217473720148213e-05, 17: 0.0008868292175658348, 18: 0.0002928634622208909, 19: 6.217339200066014e-06, 20: 0.00018134382079422958, 21: 0.002645359109055596, 22: 0.0002131156258977079, 23: 0.00014589156389685848, 24: 0.0005196190515395972, 25: 0.00035693360951943707, 26: 0.0006751699797052776, 27: 7.925509536668044e-05, 28: 3.7014857557552146e-05, 29: 1.3200870183843301e-05, 30: 6.990816089010818e-05, 31: 8.007226195926278e-06, 32: 0.00021942583528112785, 33: 0.001903126307817955, 34: 0.0001975968782896225, 35: 4.9473774019337155e-05, 36: 0.000106

In [37]:
print(f"Authorities Scores :\n{hubs_scores}")

Authorities Scores :
{0: 4.021031639777643e-05, 1: 7.319607685824188e-05, 2: 3.5017884744336435e-05, 3: 0.0010539872861763625, 4: 8.200618013274951e-05, 5: 0.00032001233330871515, 6: 0.00023181963355171467, 7: 0.00018207047608178467, 8: 0.004921182063808107, 9: 0.0002882286245176333, 10: 3.1200539556819355e-05, 11: 0.0004975871202612588, 12: 0.00041573260338093835, 13: 8.409812451935368e-05, 14: 0.00010084393686112614, 15: 3.617008656019807e-05, 16: 7.217473720148213e-05, 17: 0.0008868292175658348, 18: 0.0002928634622208909, 19: 6.217339200066014e-06, 20: 0.00018134382079422958, 21: 0.002645359109055596, 22: 0.0002131156258977079, 23: 0.00014589156389685848, 24: 0.0005196190515395972, 25: 0.00035693360951943707, 26: 0.0006751699797052776, 27: 7.925509536668044e-05, 28: 3.7014857557552146e-05, 29: 1.3200870183843301e-05, 30: 6.990816089010818e-05, 31: 8.007226195926278e-06, 32: 0.00021942583528112785, 33: 0.001903126307817955, 34: 0.0001975968782896225, 35: 4.9473774019337155e-05, 36: 0